# 新冠疫情：医疗设施容量优化

## 目标和前提条件

这个新冠疫情医疗设施容量优化问题展示了如何确定医疗设施的最佳位置和容量,以实现：

* 满足新冠患者的治疗需求,
* 最小化医疗机构开设临时设施的成本，以及
* 预测特定县的新冠患者分配到特定医疗设施的情况。

这是一个初级建模示例，我们假设你了解 Python，并且对如何构建数学优化模型有一定的了解。

**下载代码库** <br />
点击[此处](https://github.com/Gurobi/modeling-examples/archive/master.zip)可以下载包含此示例和其他示例的代码库。

---
## 问题描述

由于新冠患者数量激增，美国各县的医院正达到满负荷运转。许多医院正在考虑建立临时设施以增加其治疗新冠患者的容量。

![临时设施](covid19-trailerFacilty.jpg)

在本例中，我们关注美国的九个县。每个县都有现有的设施来治疗新冠患者，同时也可以选择建立临时设施来增加整体治疗新冠患者的容量。

下表定义了每个县的中心坐标和预测需求（即预计的新冠患者数量）。为了估计这个需求，我们考虑了加利福尼亚州九个虚构县的人口、加州当前的每日新冠病例数、需要住院的新冠病例的平均百分比，以及新冠患者的平均住院天数。

| 中心 | 坐标 | 需求 |
| --- | --- | --- |
| 县 1 | (1, 1.5) | 351 |
| 县 2 | (3, 1) | 230 |
| 县 3 | (5.5, 1.5) | 529 |
| 县 4 | (1, 4.5 ) | 339 |
| 县 5 | (3, 3.5) | 360 |
| 县 6 | (5.5, 4.5) | 527 |
| 县 7 | (1, 8) | 469 |
| 县 8 | (3, 6) | 234 |
| 县 9 | (4.5, 8) | 500 |

下表定义了现有设施的坐标和容量。现有设施的容量按所在县预测需求的80%计算。县9是个例外，我们假设其现有容量过剩。

| 现有设施 | 坐标 | 容量 |
| --- | --- | --- |
| 设施 1 | (1, 2) | 281 |
| 设施 2 | (2.5, 1) | 187 |
| 设施 3 | (5, 1) | 200 |
| 设施 4 | (6.5, 3.5) | 223 |
| 设施 5 | (1, 5) | 281 |
| 设施 6 | (3, 4) | 281 |
| 设施 7 | (5, 4) | 222 |
| 设施 8 | (6.5, 5.5) | 200 |
| 设施 9 | (1, 8.5) | 250 |
| 设施 10 | (1.5, 9.5) | 125 |
| 设施 11 | (8.5, 6) | 187 |
| 设施 12 | (5, 8) | 300 |
| 设施 13 | (3, 9) | 300 |
| 设施 14 | (6, 9) | 243 |

下表定义了新建临时设施的坐标和容量。建立一个容量为100名新冠患者的临时设施的成本为$\$500,000$。

| 临时设施 | 坐标 | 容量 |
| --- | --- | --- |
| 设施 15 | (1.5, 1) | 100 |
| 设施 16 | (3.5, 1.5) | 100 |
| 设施 17 | (5.5, 2.5) | 100 |
| 设施 18 | (1.5, 3.5) | 100 |
| 设施 19 | (3.5, 2.5) | 100 |
| 设施 20 | (4.5, 4.5) | 100 |
| 设施 21 | (1.5, 6.5) | 100 |
| 设施 22 | (3.5, 6.5) | 100 |
| 设施 23 | (5.5, 6.5) | 100 |

三个表格的坐标以十英里为单位。我们假设每增加10英里到新冠设施的距离，每名新冠患者的驾驶成本增加$\$5$。

在本例中，目标是确定要建立哪些临时设施，以便能够满足新冠患者的治疗需求，同时最小化新冠患者驾驶到现有或临时新冠设施的总成本和建立临时设施的总成本。

本例展示了一个设施选址混合整数规划（MIP）模型如何帮助医疗机构做出以下决策：

* 如何最好地利用其容量，
* 是否为新冠患者建立临时设施，以及
* 如何将县的新冠患者分配到各个医疗设施，以确保这些设施有能力为患者提供治疗。

本 Jupyter Notebook 基于 Katherine Klise 和 Michael Bynum 撰写的论文 [1]。

## 模型公式

### 集合和索引

$e \in E$: 现有医疗设施位置的索引和集合。

$t \in T$: 临时医疗设施位置的索引和集合。

$f \in F = E \cup T$: 所有医疗设施位置的索引和集合。

$c \in C$: 县的索引和集合。

### 参数

$Dist_{c,f} \in \mathbb{R}^+$: 县 $c$ 和设施位置 $f$ 之间的距离。

$Dem_{c} \in \mathbb{R}^+$: 县 $c$ 预计需要新冠设施的人数。

$Cap_{f} \in \mathbb{R}^+$: 设施位置 $f$ 可以服务的人数。

$\text{dCost} = 5$: 驾驶10英里的成本。

$\text{tFCost} = 500,000$: 建立一个容量为100名新冠患者的临时设施的成本。

$bigM$: 在临时设施增加额外容量以满足新冠患者治疗需求的惩罚。

### 决策变量

$y_{t} \in \{0, 1 \}$: 如果在位置 $t$ 建立临时设施，则该变量等于1；否则为0。

$ x_{c,f} \in \mathbb{R}^+$: 县 $c$ 的人数由位置 $f$ 的设施服务。

$z_{t} \in \mathbb{R}^+$: 在临时设施位置 $t$ 增加的额外容量。


### 目标函数

- **成本**。我们希望最小化患者从县到医疗设施的驾驶总成本和建立临时新冠治疗容量的总成本。最后一项带有大惩罚系数 ($bigM$)，允许在临时设施增加额外容量以确保满足总需求。
 

\begin{equation}
\text{Min} \quad Z = \sum_{c \in C} \sum_{f \in F} \text{dCost} *Dist_{c,f} * x_{c,f} + 
\text{tFCost}*\sum_{t \in T} y_{t} + bigM*\sum_{t \in T} z_{t}
\tag{0}
\end{equation}

### 约束条件

- **需求**。满足县对新冠设施服务的需求。

\begin{equation}
\sum_{f \in F} x_{c,f} = Dem_{c} \quad \forall c \in C
\tag{1}
\end{equation}

- **现有设施**。现有设施位置的容量不能超过。

\begin{equation}
\sum_{c \in C} x_{c,e} \leq Cap_{e} \quad \forall e \in E
\tag{2}
\end{equation}

- **临时设施**。临时设施位置的容量不能超过。请注意，可以增加额外容量。

\begin{equation}
\sum_{c \in C} x_{c,t} \leq Cap_{t}*y_{t} + z_{t} \quad \forall t \in T
\tag{3}
\end{equation}

---

## Python 实现

我们现在导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
# %pip install gurobipy

In [1]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

---

### 辅助函数

* `compute_distance` 计算县中心和设施位置之间的距离
* `solve_covid19_facility` 构建、求解并打印新冠疫情医疗设施容量优化模型的结果

In [ ]:
def compute_distance(loc1, loc2):
    
    # 此函数计算设施与县中心之间的欧几里得距离。
    
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [ ]:
def solve_covid19_facility(c_coordinates, demand):
    
    #####################################################
    #                    数据
    #####################################################
    
    # 县的索引
    counties = [*range(1,10)]
    
    # 设施的索引
    facilities = [*range(1,24)]
    
    # 创建字典来记录现有设施的坐标和治疗新冠患者的容量
    
    existing, e_coordinates, e_capacity  = gp.multidict({
        1: [(1, 2), 281],
        2: [(2.5, 1), 187],
        3: [(5, 1), 200],
        4: [(6.5, 3.5), 223],
        5: [(1, 5), 281],
        6: [(3, 4), 281],
        7: [(5, 4), 222],
        8: [(6.5, 5.5), 200],
        9: [(1, 8.5), 250], 
        10: [(1.5, 9.5), 125],
        11: [(8.5, 6), 187],
        12: [(5, 8), 300],
        13: [(3, 9), 300],
        14: [(6, 9), 243]
    })
    
    # 创建字典来记录临时设施的坐标和治疗新冠患者的容量
    
    temporary, t_coordinates, t_capacity  = gp.multidict({
        15: [(1.5, 1), 100],
        16: [(3.5, 1.5), 100],
        17: [(5.5, 2.5), 100],
        18: [(1.5, 3.5), 100],
        19: [(3.5, 2.5), 100],
        20: [(4.5, 4.5), 100],
        21: [(1.5, 6.5), 100],
        22: [(3.5, 6.5), 100],
        23: [(5.5, 6.5), 100]
    })
    
    # 驾驶10英里的成本
    dcost = 5
    
    # 建立容量为100名新冠患者的临时设施的成本
    tfcost = 500000
    
    # 计算MIP模型公式的关键参数
    f_coordinates = {}
    for e in existing:
        f_coordinates[e] = e_coordinates[e]
    
    for t in temporary:
        f_coordinates[t] = t_coordinates[t]
    
    # 县和设施的笛卡尔积
    cf = []
    
    for c in counties:
        for f in facilities:
            tp = c,f
            cf.append(tp)
        
    # 计算县中心与设施位置之间的距离
    distance = {(c,f): compute_distance(c_coordinates[c], f_coordinates[f]) for c, f in cf}
    
    #####################################################
    #                    MIP模型公式
    #####################################################
    
    m = gp.Model('covid19_temporary_facility_location')
    
    # 建立临时设施
    y = m.addVars(temporary, vtype=GRB.BINARY, name='temporary')
    
    # 将县的新冠患者分配到设施
    x = m.addVars(cf, vtype=GRB.CONTINUOUS, name='Assign')
    
    # 向临时设施增加容量
    z = m.addVars(temporary, vtype=GRB.CONTINUOUS, name='addCap' )
    
    # 目标函数：最小化驾驶到新冠设施的总距离
    
    # 在临时设施增加容量的大惩罚
    bigM = 1e9
    
    m.setObjective(gp.quicksum(dcost*distance[c,f]*x[c,f] for c,f in cf) 
                   + tfcost*y.sum()
                   + bigM*z.sum(), GRB.MINIMIZE)
    
    # 县需求约束
    demandConstrs = m.addConstrs((gp.quicksum(x[c,f] for f in facilities) == demand[c] for c in counties), 
                                 name='demandConstrs')
    
    # 现有设施容量约束
    existingCapConstrs = m.addConstrs((gp.quicksum(x[c,e]  for c in counties) <= e_capacity[e] for e in existing ), 
                                      name='existingCapConstrs')
    
    # 临时设施容量约束
    temporaryCapConstrs = m.addConstrs((gp.quicksum(x[c,t]  for c in counties) -z[t] 
                                        <= t_capacity[t]*y[t] for t in temporary ),
                                       name='temporaryCapConstrs')
    # 运行优化引擎
    m.optimize()
    
    #####################################################
    #                    输出报告
    #####################################################
    
    # 建立临时设施位置的总成本
    temporary_facility_cost = 0
    
    print(f"\n\n_____________Optimal costs______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            temporary_facility_cost += tfcost*round(y[t].x)
        
    patient_allocation_cost = 0
    for c,f in cf:
        if x[c,f].x > 1e-6:
            patient_allocation_cost += dcost*round(distance[c,f]*x[c,f].x)
            
    print(f"The total cost of building COVID-19 temporary healhtcare facilities is ${temporary_facility_cost:,}") 
    print(f"The total cost of allocating COVID-19 patients to healtcare facilities is ${patient_allocation_cost:,}")  
    
    # 在位置建立临时设施
    
    print(f"\n_____________Plan for temporary facilities______________________")
    for t in temporary:
        if (y[t].x > 0.5):
            print(f"Build a temporary facility at location {t}")
            
    # 增加临时设施的容量
    print(f"\n_____________Plan to increase Capacity at temporary Facilities______________________")
    for t in temporary:
        if (z[t].x > 1e-6):
            print(f"Increase  temporary facility capacity at location {t} by {round(z[t].x)} beds")

    # 满足每个设施的需求
    f_demand = {}
    
    print(f"\n_____________Allocation of county patients to COVID-19 healthcare facility______________________")
    for f in facilities:
        temp = 0
        for c in counties:
            allocation = round(x[c,f].x)
            if allocation > 0:
                print(f"{allocation} COVID-19 patients from county {c} are treated at facility {f} ")
            temp += allocation
        f_demand[f] = temp
        print(f"{temp} is the total number of COVID-19 patients that are treated at facility {f}. ")
        print(f"\n________________________________________________________________________________")
        
    # 测试总需求 = 设施满足的总需求
    total_demand = 0
    
    for c in counties:
        total_demand += demand[c]
        
    demand_satisfied = 0
    for f in facilities:
        demand_satisfied += f_demand[f]
        
    print(f"\n_____________Test demand = supply______________________")
    print(f"Total demand is: {total_demand:,} patients")
    print(f"Total demand satisfied is: {demand_satisfied:,} beds")

## 基本情景

在此情景中，我们考虑新冠疫情医疗设施容量优化问题实例中描述的数据。预测需求如问题描述的第一个表所定义。

In [ ]:
# 创建字典来记录县的坐标和新冠治疗需求

counties, coordinates, forecast  = gp.multidict({
    1: [(1, 1.5), 351],
    2: [(3, 1), 230],
    3: [(5.5, 1.5), 529],
    4: [(1, 4.5 ), 339],
    5: [(3, 3.5), 360],
    6: [(5.5, 4.5), 527],
    7: [(1, 8), 469],
    8: [(3, 6), 234],
    9: [(4.5, 8), 500]   
})


# 找到基本情景的最优解
solve_covid19_facility(coordinates, forecast)

Using license file c:\gurobi\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 32 rows, 225 columns and 432 nonzeros
Model fingerprint: 0xbb38e066
Variable types: 216 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Presolve time: 0.00s
Presolved: 32 rows, 225 columns, 432 nonzeros
Variable types: 216 continuous, 9 integer (9 binary)

Root relaxation: objective 1.317174e+06, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1317174.01    0    3          - 1317174.01      -     -    0s
H    0     0                    2020218.8911 1317174.01  34.8%     -    0s
H    0     0                    15

### 基本情景分析

建立新冠临时医疗设施的最优总成本为$\$1,500,000$，并建立了三个新冠临时医疗设施。将新冠患者分配到医疗设施的总成本为$\$21,645$，不需要增加额外容量来满足新冠患者的治疗需求。

MIP模型还确定了县的新冠患者分配到医疗设施的预期人数。例如，预计县3的6名新冠患者、县5的50名新冠患者和县6的166名新冠患者将在设施7接受治疗。预计在设施7接受治疗的新冠患者总数为222名。

---
## 情景1

假设疾病控制与预防中心（CDC）宣布住院人数将增加20%。这个百分比包括5%的缓冲容量，以应对预期需求的变化。

In [ ]:
# 需求增长20%

for c in counties:
    forecast[c] = round(1.2*forecast[c])
    
# 找到场景1的最优值
solve_covid19_facility(coordinates, forecast)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 32 rows, 225 columns and 432 nonzeros
Model fingerprint: 0x599a0475
Variable types: 216 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e+00, 1e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 6e+02]
Presolve time: 0.00s
Presolved: 32 rows, 225 columns, 432 nonzeros
Variable types: 216 continuous, 9 integer (9 binary)

Root relaxation: objective 6.700453e+10, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6.700453e+10 6.7005e+10  0.00%     -    0s

Explored 0 nodes (51 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 6.70045e+10 

Optimal sol

### 情景1分析

建立新冠临时医疗设施的最优总成本为$\$4,500,000$，并建立了九个新冠临时医疗设施。将新冠患者分配到医疗设施的总成本为$\$25,520$，并且需要在临时医疗设施15和17增加40和27张床位。

请注意，在此情景中，系统超负荷运转，所有新冠医疗设施都在满负荷运转。此外，还需要在一些临时医疗设施增加额外容量。

---
## 结论

在本例中，我们解决了新冠疫情医疗设施容量优化问题。我们确定了医疗设施的最佳位置和容量，以实现：
* 满足新冠患者的治疗需求，
* 最小化医疗机构开设临时设施的成本，以及
* 预测特定县的新冠患者分配到特定医疗设施的情况。

我们探讨了两种情景。在基本情景中，我们有足够的容量，需要建立三个临时医疗设施。而在另一种情景（情景1）中，随着需要住院的新冠患者数量增加20%，我们需要建立九个临时医疗设施，并在其中两个增加额外容量。

我们的新冠疫情医疗设施选址优化模型可以帮助公共卫生官员和医疗机构在新冠疫情期间做出关于何时何地增加医疗设施容量的战略决策。此外，这个战略模型可以为一个新冠负载平衡调度模型提供信息，该模型能够（实时）将需要住院的新冠患者分配到“合适”的医疗设施。

此外，我们的模型可以为一个战术模型提供信息，该模型确定如何增加容量以应对需求的增加。例如，雇用、培训和重新技能化的医疗人员数量，医疗人员的轮换，以及所需设备（如呼吸机、药物、床位等）的数量。

## 参考文献
[1] Katherine Klise 和 Michael Bynum. *Facility Location Optimization Model for COVID-19 Resources*. 2020年4月. 联合DOE实验室大流行建模和分析能力. SAND2020-4693R.

版权所有 © 2020 Gurobi Optimization, LLC